In [1]:
import asyncio
import os
from dotenv import load_dotenv
from omnipok_agent.agents.plan_solve_agent import PlanSolveAgent
from omnipok_agent.core import BaseAgent, RunContext
from omnipok_agent.tools import global_registry, web_search, http_get
from omnipok_agent.llm import OmniPokLLM
from omnipok_agent.memory import InMemoryMemory

# 加载环境变量
load_dotenv()

async def main():
    try:
        # 注册工具
        global_registry.register(tool=web_search)
        global_registry.register(tool=http_get)
        
        # 检查工具是否注册成功
        available_tools = global_registry.list_tools()
        print(f"已注册的工具数量: {len(available_tools)}")
        for tool in available_tools:
            print(f"  - {tool.name}: {tool.description[:50]}...")
        
        # 创建带工具的 Agent
        # 重要：需要设置 system_prompt 告诉 Agent 可以使用工具
        agent = PlanSolveAgent(
            agent_id="tool-agent",
            name="工具助手",
            llm=OmniPokLLM(),
            memory=InMemoryMemory(),
            tool_registry=global_registry,
            system_prompt="""你是一个可以使用工具来帮助用户的AI助手。
当用户需要实时信息、搜索网络、访问API或执行其他操作时，你必须使用提供的工具来完成任务。

重要：当用户要求你搜索信息或访问URL时，你必须使用工具调用（tool_calls），而不是用文字描述。

可用的工具包括：
- web_search: 用于搜索网络获取实时信息，参数：query (搜索关键词)
- http_get: 用于访问HTTP API获取数据，参数：url (要访问的URL)

请使用工具调用来完成任务，不要只是告诉用户你无法完成。"""
        )
        
        context = RunContext(tenant_id="t1", user_id="u1")
        # Agent 现在可以使用 web_search 和 http_get 工具
        response = await agent.process(
            "请搜索一下Python的最新版本是什么",
            context
        )
        print("\n=== Agent 响应 ===")
        print(response)
    except Exception as e:
        print(f"错误: {e}")
        import traceback
        traceback.print_exc()
        print("\n提示：")
        print("- 请确保已配置 LLM API 密钥（如 OPENAI_API_KEY）")
        print("- web_search 工具需要 SERPAPI_API_KEY（可选）")

# 运行主函数
await main()


已注册的工具数量: 2
  - web_search: 一个基于SerpApi的实战网页搜索引擎工具。
它会智能地解析搜索结果，优先返回直接答案或知识图谱信...
  - http_get: Make an HTTP GET request.

Args:
    url: The URL ...

=== Agent 响应 ===
执行计划：
1. 使用 web_search 工具，搜索关键词 "Python 最新版本"
